# Evaluate a linear regression model using HoldOut Validation

![Holdout](https://miro.medium.com/max/720/1*0aJyXdHtRrEejuWUWFM07g.webp)

# Import Libs

In [58]:
import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Global Variables

In [3]:
SEED = 42

In [25]:
TEST_SIZE = 0.2
VALID_SIZE = 0.2

# Load dataset

In [4]:
# Load the Boston Dataset
bhp = datasets.load_boston()

C:\Users\Emerson\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset

In [14]:
pd.DataFrame(bhp.data, columns=bhp.feature_names)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


# Create Test Split

In [24]:
X = bhp.data
y = bhp.target

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=TEST_SIZE, 
                                                    shuffle=True,
                                                    random_state=SEED)

# Create Training and Validation Split

In [28]:
# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train, 
                                                  test_size=VALID_SIZE, 
                                                  random_state=SEED)

In [31]:
print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))
print("X_val shape: {}".format(X_val.shape))
print("y_val shape: {}".format(y_val.shape))

X_train shape: (323, 13)
y_train shape: (323,)
X_test shape: (102, 13)
y_test shape: (102,)
X_val shape: (81, 13)
y_val shape: (81,)


# Create model

## Model I

In [32]:
model_linear = LinearRegression()

## Model II

In [59]:
model_lasso = Lasso(alpha=0.1)

## Model III

In [60]:
model_tree = DecisionTreeRegressor()

# Evaluate model using Holdout Validation

## Choosing best model using Validation

In [68]:
def get_metrics(list_models, X_train, y_train, X_test, y_test):
    
    # INIT DICT RESULT
    dict_result = {}
    
    for model in list_models:

        # LEARNING WITH TRAIN
        model.fit(X_train, y_train)
        
        # PREDICT NEW INSTANCES
        predicts = model.predict(X_test)
        
        # CALCULATE COST FUNCTION
        error = np.sqrt(mean_squared_error(y_true=y_val, y_pred=predicts))
        
        # SAVE RESULT
        dict_result[model] = {"rmse": error}
        
    return dict_result

In [69]:
result_models = get_metrics(list_models=[model_linear, model_lasso, model_tree], 
                                         X_train=X_train, 
                                         y_train=y_train, 
                                         X_test=X_val, 
                                         y_test=y_val)

In [70]:
result_models

{LinearRegression(): {'rmse': 4.825602351498199},
 Lasso(alpha=0.1): {'rmse': 4.802435126536561},
 DecisionTreeRegressor(): {'rmse': 6.64529909033446}}